In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 


from Artha.nlp_extraction import *
import Artha.data_process as dp
# from Artha.neocoinsdb import Neo
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import csv
import json
from collections import Counter
import numpy as np
np.set_printoptions(suppress=True)
import pprint
import math
from datetime import datetime

pp = pprint.PrettyPrinter(indent=1)

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')

CPU times: user 2.12 s, sys: 315 ms, total: 2.44 s
Wall time: 3.6 s


In [ ]:
# neo.session.run('''
#                 Merge (n:Person {username: 'checkra_'})
#                 ON CREATE SET n.id ='1356259499431129092',
#                 n.name = 'Checkra',
#                 n.username = 'checkra_'
#         ''')

In [11]:
print(len(neo.get_nodes()), len(neo.get_relations()))

771 8823


In [6]:
neo.clear_db()

deleted database


In [8]:
neo.load_coins()
neo.load_follow_nodes(dp.load_following("checkra_"))

Updated coins
loaded People


In [3]:
# add follows between only people I follow, can change by getting rid of second 2 last line for anyone a person follows
follows = []
checkra_follows = [i[3] for i in dp.load_following("checkra_")]
for file_loc in os.listdir(path = "../data/follows/"):
    if file_loc != "ucheckra_.csv":
        fs = dp.load_following(file_loc[1:-4])
        for i in fs:
            if i[3] in checkra_follows:
                follows.append(i)

In [10]:
%%time
print(len(follows))
neo.load_follow_nodes(follows)
neo.load_follow_relations(follows)

8823
loaded People
loaded relations
CPU times: user 44.1 ms, sys: 4.27 ms, total: 48.4 ms
Wall time: 3.13 s


In [ ]:
follow_path = "../data/follows/"
tweets_path = "../data/tweets/"

l1, l2 = os.listdir(follow_path), os.listdir(tweets_path)
l1 = [i[1:-4] for i in l1]
l2 = [i[1:-11] for i in l2]
print(len(l1), len(l2))
both = np.intersect1d(l1, l2)
print(len(both))
for person in both[:1]:
    # tweets = dp.load_tweets(person)
    # docs = run_pipeline(person)
    # following = dp.
    for doc in docs:
        if doc._.tickers and "BTC" not in doc._.tickers and "USD" not in doc._.tickers and "ETF" not in doc._.tickers:
            print(doc, doc.ents, doc._.tickers, "\n\n")
            # break
# TODO CLEAN UP ENTITY RECOGNITION

In [ ]:
names = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
all_mentions = []

In [ ]:
%%time
for ind, name in enumerate(names):
    # tweets = dp.load_tweets(name)
    docs = run_pipeline(name)
    # print(tweets[0]["created_at"])
    print(ind, name)
    mentions = get_mention_scores(name, docs)
    all_mentions.extend(mentions)

In [ ]:
%%time
neo.load_mentions(all_mentions)

In [ ]:
name = "BTC_JackSparrow"
docs = run_pipeline(name)
    # print(tweets[0]["created_at"])
print(name)
singlementions = get_mention_scores(name, docs)

In [11]:
for i in singlementions:
    print(i)

('BTC_JackSparrow', 'BTC', 11.88777)
('BTC_JackSparrow', 'ETH', 3.36575)
('BTC_JackSparrow', 'LINA', 1.07525)
('BTC_JackSparrow', 'UTK', 0.56272)
('BTC_JackSparrow', 'DOGE', 1.14677)
('BTC_JackSparrow', 'USD', 0.7647)
('BTC_JackSparrow', 'BNB', 0.01759)
('BTC_JackSparrow', 'ETF', 0.35056)
('BTC_JackSparrow', 'WRX', 0.13727)
('BTC_JackSparrow', 'LTC', 0.13315)
('BTC_JackSparrow', 'UNI', 0.18669)
('BTC_JackSparrow', 'AVAX', 0.17863)
('BTC_JackSparrow', 'CRV', 0.14257)
('BTC_JackSparrow', 'XVG', 0.02611)
('BTC_JackSparrow', 'SCRT', 0.02807)
('BTC_JackSparrow', 'USDT', 0.00168)
('BTC_JackSparrow', 'DAI', 0.12401)
('BTC_JackSparrow', 'MKR', 0.12401)
('BTC_JackSparrow', 'WINGS', 0.11663)
('BTC_JackSparrow', 'HNST', 0.04438)
('BTC_JackSparrow', 'XRP', 0.00854)
('BTC_JackSparrow', 'DENT', 0.00818)
('BTC_JackSparrow', 'COMP', 0.00644)
('BTC_JackSparrow', 'CAN', 0.00609)
('BTC_JackSparrow', 'SUN', 0.00361)
('BTC_JackSparrow', 'TRX', 0.00199)
('BTC_JackSparrow', 'FTT', 0.00154)
('BTC_JackSparrow'

In [ ]:
for doc in docs:
    if "WRX" in doc._.tickers:
        print(doc, doc._.tweeted_at)

In [ ]:
%%time
neo.load_mentions(singlementions)

In [15]:
result = neo.session.run('''
    CALL gds.graph.create(
        'mentions',
        {Person: {label: "Person"},Coin: {label: "Coin"}},
        {CALLOUT: {type : 'CALLOUT', orientation: 'NATURAL'}},
        { relationshipProperties: 'weight'}
    )
    YIELD graphName, nodeCount, relationshipCount
    ''')
[r for r in result]

[<Record graphName='mentions' nodeCount=771 relationshipCount=26>]

In [26]:
result = neo.session.run('''
    CALL gds.pageRank.stream('mentions', 
        {
        maxIterations: 20,
        dampingFactor: 0.85,
        relationshipWeightProperty: 'weight'
        })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).ticker AS ticker, score
    ORDER BY score DESC, ticker ASC

''')
f = [r for r in result]
for i in f:
    if i["score"]>.1500001:
        print(i)

<Record ticker='BTC' score=0.2285871043801308>
<Record ticker='ETH' score=0.17225014148280027>
<Record ticker='DOGE' score=0.15758101278916004>
<Record ticker='LINA' score=0.15710821207612755>
<Record ticker='UTK' score=0.1537200027611107>
<Record ticker='UNI' score=0.15123416140559132>
<Record ticker='AVAX' score=0.15118087876471692>
<Record ticker='CRV' score=0.15094249499379658>
<Record ticker='WRX' score=0.15090745800407607>
<Record ticker='LTC' score=0.15088022167910825>
<Record ticker='DAI' score=0.15081979942915497>
<Record ticker='MKR' score=0.15081979942915497>
<Record ticker='WINGS' score=0.15077101207571106>
<Record ticker='SCRT' score=0.15018556382201498>
<Record ticker='XVG' score=0.1501726067523123>
<Record ticker='BNB' score=0.15011628313659459>
<Record ticker='XRP' score=0.1500564558293263>
<Record ticker='DENT' score=0.15005407595963335>
<Record ticker='COMP' score=0.15004257324580977>
<Record ticker='SUN' score=0.15002386481764918>
<Record ticker='TRX' score=0.1500131

In [14]:
result = neo.session.run("CALL gds.graph.drop('mentions') YIELD graphName")
[r for r in result]

[<Record graphName='mentions'>]

In [ ]:
# make graphview that only looks at people who mention coins, and coins that are mentioned

# exchange tokens might have more mentions
# factor in mentions of coins in threads, so it gets more mentions than just 1